<h1>Set 2</h1>
<h5>Challenge 9</h5>
<p>Implement PKCS#7 padding</p>

In [110]:
def pkcs7padding(plaintext, blocksize):
    size = len(plaintext)
    numBlocks = size // blocksize + 1
    numPads = blocksize % size
    return plaintext + bytes2str(numPads * (numPads).to_bytes(1, byteorder='little'))

plaintext = "YELLOW SUBMARINE"
result = "YELLOW SUBMARINE\x04\x04\x04\x04"
pkcs7padding(plaintext, 20) == result

True

<h5>Challenge 10</h5>
<p>Implement CBC</p>

In [ ]:
from crypto import *

class CBC:
    def __init__(self, key, iv):
        self.key = key
        self.iv = iv
    def encrypt(self,plaintext):
        
        
        ciphertext = XOR(str2bytes(plaintext), self.key)
        
        ciphertext = 
        return plaintext
    def decrypt(self,plaintext):
        return ciphertext

In [ ]:
'this is a text'